In [17]:
# import os
# import glob
# import numpy as np
# import pandas as pd

# # ============================================================
# # USER INPUTS
# # ============================================================
# BASE_DIR   = r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1_copy_data_5"
# MECHS      = ["plm", "psm", "psmi"]
# GRID_SIZE  = 40  # grid40 => 40x40 = 1600 location_ids

# # Which columns to discretize (true lat/lon -> location_id)
# LAT_COL = "latitude"
# LON_COL = "longitude"

# # Output column name
# LOC_ID_COL = "location_id"

# # If you want consistent discretization across ALL files in a mechanism folder,
# # keep this True. (Recommended for ML: same lat/lon always maps to same id.)
# GLOBAL_BOUNDS_PER_MECH = True

# # ============================================================
# # HELPERS
# # ============================================================
# def compute_bounds(csv_paths, lat_col, lon_col):
#     """Compute global min/max across many CSVs (robust: ignores NaNs)."""
#     lat_min = np.inf
#     lat_max = -np.inf
#     lon_min = np.inf
#     lon_max = -np.inf

#     for p in csv_paths:
#         df = pd.read_csv(p, usecols=[lat_col, lon_col])
#         lat_min = min(lat_min, np.nanmin(df[lat_col].to_numpy()))
#         lat_max = max(lat_max, np.nanmax(df[lat_col].to_numpy()))
#         lon_min = min(lon_min, np.nanmin(df[lon_col].to_numpy()))
#         lon_max = max(lon_max, np.nanmax(df[lon_col].to_numpy()))

#     # Handle edge case: all values equal (avoid zero step)
#     if lat_min == lat_max:
#         lat_max = lat_min + 1e-12
#     if lon_min == lon_max:
#         lon_max = lon_min + 1e-12

#     return lat_min, lat_max, lon_min, lon_max


# def latlon_to_location_id(lat, lon, lat_min, lat_max, lon_min, lon_max, grid_size):
#     """
#     Map (lat, lon) -> integer location_id in [1, grid_size^2]
#     Uses uniform grid, clamps boundary points safely.
#     """
#     lat_step = (lat_max - lat_min) / grid_size
#     lon_step = (lon_max - lon_min) / grid_size

#     lat_pos = np.floor((lat - lat_min) / lat_step).astype(int)
#     lon_pos = np.floor((lon - lon_min) / lon_step).astype(int)

#     # Clamp to [0, grid_size-1]
#     lat_pos = np.clip(lat_pos, 0, grid_size - 1)
#     lon_pos = np.clip(lon_pos, 0, grid_size - 1)

#     # 2D -> 1D, +1 to make IDs start at 1
#     return (lat_pos * grid_size + lon_pos + 1).astype(int)


# def add_location_id(df, lat_col, lon_col, grid_size, bounds):
#     lat_min, lat_max, lon_min, lon_max = bounds

#     lat = df[lat_col].to_numpy(dtype=float)
#     lon = df[lon_col].to_numpy(dtype=float)

#     df[LOC_ID_COL] = latlon_to_location_id(lat, lon, lat_min, lat_max, lon_min, lon_max, grid_size)
#     return df


# # ============================================================
# # MAIN
# # ============================================================
# for mech in MECHS:
#     mech_dir = os.path.join(BASE_DIR, mech)
#     if not os.path.isdir(mech_dir):
#         print(f"[SKIP] Missing mech dir: {mech_dir}")
#         continue

#     # Collect ALL csv files in mech folder (only files directly under plm/psm/psmi)
#     # If you also want to include nested folders, switch to: glob.glob(mech_dir + "/**/*.csv", recursive=True)
#     csv_paths = sorted(glob.glob(os.path.join(mech_dir, "*.csv")))

#     if not csv_paths:
#         print(f"[SKIP] No CSVs in: {mech_dir}")
#         continue

#     # Output folder: plm/grid40 (or psm/grid40, psmi/grid40)
#     out_dir = os.path.join(mech_dir, f"grid{GRID_SIZE}")
#     os.makedirs(out_dir, exist_ok=True)

#     # Bounds: either global per mechanism (recommended) or per file
#     if GLOBAL_BOUNDS_PER_MECH:
#         bounds = compute_bounds(csv_paths, LAT_COL, LON_COL)
#         print(f"[INFO] {mech}: global bounds lat[{bounds[0]}, {bounds[1]}], lon[{bounds[2]}, {bounds[3]}]")

#     for p in csv_paths:
#         df = pd.read_csv(p)

#         # Basic safety checks
#         if LAT_COL not in df.columns or LON_COL not in df.columns:
#             print(f"[SKIP] Missing {LAT_COL}/{LON_COL} in {p}")
#             continue

#         if not GLOBAL_BOUNDS_PER_MECH:
#             bounds = (df[LAT_COL].min(), df[LAT_COL].max(), df[LON_COL].min(), df[LON_COL].max())
#             # avoid zero step
#             if bounds[0] == bounds[1]:
#                 bounds = (bounds[0], bounds[1] + 1e-12, bounds[2], bounds[3])
#             if bounds[2] == bounds[3]:
#                 bounds = (bounds[0], bounds[1], bounds[2], bounds[3] + 1e-12)

#         df = add_location_id(df, LAT_COL, LON_COL, GRID_SIZE, bounds)

#         base = os.path.splitext(os.path.basename(p))[0]
#         out_name = f"{base}_grid{GRID_SIZE}.csv"
#         out_path = os.path.join(out_dir, out_name)

#         df.to_csv(out_path, index=False)
#         print(f"[OK] {mech}: {os.path.basename(p)} -> {out_name}")

# print("Done.")


In [18]:
# import os
# import glob
# import numpy as np
# import pandas as pd
# from collections import defaultdict
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# # ============================================================
# # CONFIG (EDIT THESE)
# # ============================================================
# PLM_CSV  = r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1_copy_data_5\plm\grid200\merged_geotrace_plm_delta_3_Noisy_threshold_5_eps05_grid200.csv"
# PSM_CSV  = r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1_copy_data_5\psm\grid200\merged_geotrace_psm_delta_3_Noisy_threshold_5_eps05_grid200.csv"
# PSMI_CSV = r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1_copy_data_5\psmi\grid200\merged_geotrace_psmi_delta_3_Noisy_threshold_5_eps05_grid200.csv"

# # Column names in your merged CSV
# ID_COL   = "identifier"  # trajectory/user id column (change if yours differs)
# Y_COL    = "location_id"
# X_COLS   = ["perturbed_latitude", "perturbed_longitude"]  # attacker observes these

# # Evaluation settings
# TEST_SIZE          = 0.20
# RANDOM_SEED        = 42
# MAX_TRACE_LEN_LIST = [1, 5]  # edit as you like
# HISTORY_R          = 5      # r in your paper
# ALPHA_DECAY        = 0.7    # exponential decay for weights w_j ~ alpha^(j-1)
# LAPLACE_SMOOTH     = 1e-6   # smoothing for transition probs to avoid zeros
# USE_STANDARDIZE    = True   # recommended for kNN on lat/lon

# # ============================================================
# # HELPERS
# # ============================================================
# def make_weights(r: int, alpha: float) -> np.ndarray:
#     """Exponential decay weights summing to 1: w_j for j=1..r."""
#     w = np.array([alpha ** (j - 1) for j in range(1, r + 1)], dtype=float)
#     w /= w.sum()
#     return w  # shape (r,)

# def split_by_trajectory(df: pd.DataFrame, id_col: str, test_size: float, seed: int):
#     """Split train/test by whole trajectories (no leakage across same identifier)."""
#     ids = df[id_col].dropna().unique()
#     rng = np.random.default_rng(seed)
#     rng.shuffle(ids)
#     n_test = int(np.ceil(len(ids) * test_size))
#     test_ids = set(ids[:n_test])
#     train_mask = ~df[id_col].isin(test_ids)
#     test_mask  = df[id_col].isin(test_ids)
#     return df[train_mask].copy(), df[test_mask].copy()

# def build_transition_matrix(train_df: pd.DataFrame, id_col: str, y_col: str, smooth: float = 1e-6):
#     """
#     Build transition probability matrix P(next | prev) from training trajectories.
#     Returns:
#       T: dict prev_state -> (next_states array, probs array)
#       all_states: sorted unique states
#       prior: global prior over states (fallback)
#     """
#     # All states (for fallback / smoothing)
#     all_states = np.sort(train_df[y_col].dropna().unique()).astype(int)
#     state_to_idx = {s: i for i, s in enumerate(all_states)}
#     K = len(all_states)

#     # Count transitions
#     counts = np.zeros((K, K), dtype=float)  # counts[prev_idx, next_idx]
#     # Also state prior (frequency)
#     prior_counts = np.zeros(K, dtype=float)

#     # Ensure correct order within each trajectory: assume merged CSV is already in time order.
#     # If you have a timestamp column, sort within each group before counting.
#     for tid, g in train_df.groupby(id_col, sort=False):
#         seq = g[y_col].to_numpy()
#         # drop NaNs
#         seq = seq[~pd.isna(seq)].astype(int)
#         if len(seq) == 0:
#             continue
#         for s in seq:
#             if s in state_to_idx:
#                 prior_counts[state_to_idx[s]] += 1
#         for i in range(len(seq) - 1):
#             a, b = seq[i], seq[i + 1]
#             if a in state_to_idx and b in state_to_idx:
#                 counts[state_to_idx[a], state_to_idx[b]] += 1

#     # Smooth + row-normalize
#     counts += smooth
#     row_sums = counts.sum(axis=1, keepdims=True)
#     probs = counts / row_sums

#     # Build sparse dict view for speed
#     T = {}
#     for prev_idx, prev_state in enumerate(all_states):
#         T[prev_state] = (all_states, probs[prev_idx])  # full support (simple + stable)

#     # Global prior (fallback)
#     prior_counts += smooth
#     prior = prior_counts / prior_counts.sum()

#     return T, all_states, prior

# def train_knn_classifier(train_df: pd.DataFrame, x_cols, y_col: str):
#     X = train_df[x_cols].to_numpy(dtype=float)
#     y = train_df[y_col].to_numpy(dtype=int)

#     scaler = None
#     if USE_STANDARDIZE:
#         scaler = StandardScaler()
#         X = scaler.fit_transform(X)

#     k = max(1, int(round(np.log(max(len(X), 2)))))  # safe log
#     knn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1, weights="distance")
#     knn.fit(X, y)
#     return knn, scaler, k

# def get_knn_proba(knn: KNeighborsClassifier, scaler, x_cols, df_row: pd.Series, all_states: np.ndarray):
#     """
#     Return probability vector over all_states aligned to all_states.
#     Uses knn.predict_proba and aligns class order.
#     """
#     x = df_row[x_cols].to_numpy(dtype=float).reshape(1, -1)
#     if scaler is not None:
#         x = scaler.transform(x)

#     proba = knn.predict_proba(x)[0]           # aligned to knn.classes_
#     classes = knn.classes_.astype(int)

#     # Map to full state space
#     p_full = np.zeros(len(all_states), dtype=float)
#     class_to_pos = {c: i for i, c in enumerate(classes)}
#     for i, s in enumerate(all_states):
#         if s in class_to_pos:
#             p_full[i] = proba[class_to_pos[s]]
#         else:
#             p_full[i] = 0.0

#     # Numerical safety
#     ssum = p_full.sum()
#     if ssum <= 0:
#         p_full[:] = 1.0 / len(all_states)
#     else:
#         p_full /= ssum
#     return p_full  # shape (K,)

# def predict_trace_sequence(test_trace: pd.DataFrame,
#                            knn, scaler,
#                            x_cols, y_col, id_col,
#                            T, all_states, prior,
#                            r: int, weights: np.ndarray):
#     """
#     Predict a sequence of location_id for one trajectory using:
#       score(s) ∝ Pr(s|o_t) * Pr(s|H_t)
#     where Pr(s|H_t) = sum_{j=1..r} w_j * Pr(s | s_{t-j})
#     and Pr(s | s_prev) from transition matrix T.

#     Returns:
#       y_true (np.ndarray), y_pred (np.ndarray)
#     """
#     y_true = test_trace[y_col].to_numpy(dtype=int)
#     preds = []

#     # Pre-extract transition rows for speed
#     # T[prev] gives (all_states, probs_prev->*)
#     def trans_from(prev_state: int):
#         if prev_state in T:
#             return T[prev_state][1]  # probs over all_states
#         return prior  # fallback

#     for t in range(len(test_trace)):
#         p_obs = get_knn_proba(knn, scaler, x_cols, test_trace.iloc[t], all_states)

#         if t == 0 or len(preds) == 0:
#             # initial prediction: argmax Pr(s|o1)
#             s_hat = all_states[int(np.argmax(p_obs))]
#             preds.append(int(s_hat))
#             continue

#         # History-based transition mixture
#         p_hist = np.zeros_like(prior, dtype=float)
#         # use up to r previous predictions
#         for j in range(1, r + 1):
#             if t - j < 0:
#                 break
#             prev_pred = preds[t - j]
#             wj = weights[j - 1]
#             p_hist += wj * trans_from(prev_pred)

#         # If we had less than r history, renormalize weights effectively used
#         if t < r:
#             w_used = weights[:t].sum()
#             if w_used > 0:
#                 p_hist /= w_used
#             else:
#                 p_hist = prior.copy()

#         # Combine
#         score = p_obs * p_hist
#         if score.sum() <= 0:
#             score = p_obs  # fallback
#         s_hat = all_states[int(np.argmax(score))]
#         preds.append(int(s_hat))

#     return y_true, np.array(preds, dtype=int)

# def bayes_risk_vs_trace_len(df: pd.DataFrame, name: str):
#     """
#     1) Split by trajectories
#     2) Train kNN on train
#     3) Build transition matrix on train (true location_id sequences)
#     4) For each L in MAX_TRACE_LEN_LIST, evaluate Bayes risk on test trajectories
#        using first L points of each trajectory (if trajectory length >= L).
#     """
#     # Basic checks
#     for c in [ID_COL, Y_COL] + X_COLS:
#         if c not in df.columns:
#             raise ValueError(f"[{name}] Missing required column: {c}")

#     # Split by trajectory to avoid leakage
#     train_df, test_df = split_by_trajectory(df, ID_COL, TEST_SIZE, RANDOM_SEED)

#     # Train classifier
#     knn, scaler, k = train_knn_classifier(train_df, X_COLS, Y_COL)
#     print(f"[{name}] kNN k={k}, train_rows={len(train_df)}, test_rows={len(test_df)}")

#     # Transition matrix on training data
#     T, all_states, prior = build_transition_matrix(train_df, ID_COL, Y_COL, smooth=LAPLACE_SMOOTH)

#     weights = make_weights(HISTORY_R, ALPHA_DECAY)

#     # Group test traces
#     test_groups = list(test_df.groupby(ID_COL, sort=False))

#     results = []
#     for L in MAX_TRACE_LEN_LIST:
#         correct = 0
#         total = 0
#         used_traces = 0

#         for tid, g in test_groups:
#             if len(g) < L:
#                 continue
#             used_traces += 1
#             gL = g.iloc[:L]  # first L points in order

#             y_true, y_pred = predict_trace_sequence(
#                 gL, knn, scaler, X_COLS, Y_COL, ID_COL,
#                 T, all_states, prior, HISTORY_R, weights
#             )

#             correct += int((y_true == y_pred).sum())
#             total += len(y_true)

#         if total == 0:
#             results.append(np.nan)
#         else:
#             acc = correct / total
#             risk = 1.0 - acc
#             results.append(risk)

#         print(f"[{name}] L={L:>3}  traces_used={used_traces:>4}  points={total:>6}  BayesRisk={results[-1]}")
#     return results

# # ============================================================
# # RUN FOR PLM / PSM / PSMI
# # ============================================================
# def load_csv(path: str) -> pd.DataFrame:
#     return pd.read_csv(path)

# if __name__ == "__main__":
#     df_plm  = load_csv(PLM_CSV)
#     df_psm  = load_csv(PSM_CSV)
#     df_psmi = load_csv(PSMI_CSV)

#     risks_plm  = bayes_risk_vs_trace_len(df_plm,  "PLM")
#     risks_psm  = bayes_risk_vs_trace_len(df_psm,  "PSM")
#     risks_psmi = bayes_risk_vs_trace_len(df_psmi, "PSM-I")

#     print("\n==== FINAL LISTS (aligned with MAX_TRACE_LEN_LIST) ====")
#     print("trace_lengths =", MAX_TRACE_LEN_LIST)
#     print("plm  =", risks_plm)
#     print("psm  =", risks_psm)
#     print("psmi =", risks_psmi)


In [19]:
# import os
# import numpy as np
# import pandas as pd
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.preprocessing import StandardScaler

# # ============================================================
# # EDIT THESE PATHS
# # ============================================================
# PLM_CSV  = r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1_copy_data_5\plm\grid40\merged_geotrace_plm_delta_3_Noisy_threshold_5_eps05_grid40.csv"
# PSM_CSV  = r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1_copy_data_5\psm\grid40\merged_geotrace_psm_delta_3_Noisy_threshold_5_eps05_grid40.csv"
# PSMI_CSV = r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1_copy_data_5\psmi\grid40\merged_geotrace_psmi_delta_3_Noisy_threshold_5_eps05_grid40.csv"


# # ============================================================
# # COLUMN NAMES (EDIT IF NEEDED)
# # ============================================================
# ID_COL = "identifier"  # tells which trace a row belongs to
# X_COLS = ["perturbed_latitude", "perturbed_longitude"]  # attacker observes
# Y_COL  = "location_id"  # attacker wants to predict

# # ============================================================
# # SETTINGS
# # ============================================================
# TEST_RATIO = 0.20
# SEED = 42

# TRACE_LENGTHS = [1, 5, 10, 20, 25]  # L values you want

# # How many previous predicted states to use (history length)
# # For trace length L, we use min(R, L-1) previous predictions
# R = 10

# # Transition smoothing (avoid zero-prob transitions)
# SMOOTH = 1e-6

# # Use distance-weighted kNN and standardize lat/lon (recommended)
# USE_STANDARDIZE = True


# # ============================================================
# # HELPERS
# # ============================================================
# def split_by_identifier(df, id_col, test_ratio, seed):
#     """Split so that entire traces (identifiers) are either train or test (no leakage)."""
#     ids = df[id_col].dropna().unique()
#     rng = np.random.default_rng(seed)
#     rng.shuffle(ids)
#     n_test = int(np.ceil(len(ids) * test_ratio))
#     test_ids = set(ids[:n_test])
#     train_df = df[~df[id_col].isin(test_ids)].copy()
#     test_df  = df[df[id_col].isin(test_ids)].copy()
#     return train_df, test_df


# def train_knn(train_df):
#     X = train_df[X_COLS].to_numpy(dtype=float)
#     y = train_df[Y_COL].to_numpy(dtype=int)

#     scaler = None
#     if USE_STANDARDIZE:
#         scaler = StandardScaler()
#         X = scaler.fit_transform(X)

#     k = max(1, int(round(np.log(max(len(X), 2)))))
#     knn = KNeighborsClassifier(n_neighbors=k, weights="distance", n_jobs=-1)
#     knn.fit(X, y)
#     return knn, scaler, k


# def knn_proba_full(knn, scaler, x_row, all_states):
#     """Return Pr(s|o) aligned to all_states."""
#     x = x_row.reshape(1, -1)
#     if scaler is not None:
#         x = scaler.transform(x)

#     p = knn.predict_proba(x)[0]                # aligned to knn.classes_
#     classes = knn.classes_.astype(int)

#     p_full = np.zeros(len(all_states), dtype=float)
#     cls_to_idx = {c: i for i, c in enumerate(classes)}
#     for i, s in enumerate(all_states):
#         if s in cls_to_idx:
#             p_full[i] = p[cls_to_idx[s]]

#     ssum = p_full.sum()
#     if ssum <= 0:
#         p_full[:] = 1.0 / len(all_states)
#     else:
#         p_full /= ssum
#     return p_full


# def build_transition_probs(train_df):
#     """
#     Build Pr(next_state | prev_state) from training TRUE location_id sequences.
#     Output:
#       all_states: sorted list of states
#       P: dict prev_state -> prob vector over all_states
#       prior: global prior over states (fallback)
#     """
#     all_states = np.sort(train_df[Y_COL].dropna().unique()).astype(int)
#     idx = {s: i for i, s in enumerate(all_states)}
#     K = len(all_states)

#     counts = np.zeros((K, K), dtype=float)
#     prior_counts = np.zeros(K, dtype=float)

#     for _, g in train_df.groupby(ID_COL, sort=False):
#         seq = g[Y_COL].to_numpy()
#         seq = seq[~pd.isna(seq)].astype(int)
#         if len(seq) == 0:
#             continue

#         for s in seq:
#             if s in idx:
#                 prior_counts[idx[s]] += 1

#         for t in range(len(seq) - 1):
#             a, b = seq[t], seq[t + 1]
#             if a in idx and b in idx:
#                 counts[idx[a], idx[b]] += 1

#     # Smooth and normalize
#     counts += SMOOTH
#     row_sums = counts.sum(axis=1, keepdims=True)
#     probs = counts / row_sums

#     prior_counts += SMOOTH
#     prior = prior_counts / prior_counts.sum()

#     P = {prev: probs[idx[prev]] for prev in all_states}
#     return all_states, P, prior


# def history_transition_mix(P, prior, all_states, prev_preds, r):
#     """
#     Compute Pr(s | history) from previous predicted states.
#     Here we use a simple decaying weight (recent history matters more).
#     """
#     if len(prev_preds) == 0:
#         return prior.copy()

#     use = prev_preds[-r:]  # last r preds
#     # exponential decay weights: most recent gets largest
#     alpha = 0.7
#     w = np.array([alpha**i for i in range(len(use))], dtype=float)  # i=0 is most recent
#     w = w / w.sum()

#     p_hist = np.zeros(len(all_states), dtype=float)
#     # combine transitions from each previous predicted state
#     for weight, prev_state in zip(w, reversed(use)):  # reversed so most recent aligns with i=0
#         if prev_state in P:
#             p_hist += weight * P[prev_state]
#         else:
#             p_hist += weight * prior

#     # normalize
#     ssum = p_hist.sum()
#     if ssum <= 0:
#         return prior.copy()
#     return p_hist / ssum


# def bayes_risk_vs_trace_length(df, name):
#     # sanity
#     for c in [ID_COL, Y_COL] + X_COLS:
#         if c not in df.columns:
#             raise ValueError(f"[{name}] Missing column: {c}")

#     # split by trajectory
#     train_df, test_df = split_by_identifier(df, ID_COL, TEST_RATIO, SEED)

#     # train models
#     knn, scaler, k = train_knn(train_df)
#     all_states, P, prior = build_transition_probs(train_df)

#     print(f"[{name}] train_rows={len(train_df)} test_rows={len(test_df)}  kNN_k={k}  #states={len(all_states)}")

#     # group test trajectories (keep order as-is; assumes merged is time-ordered within identifier)
#     test_groups = list(test_df.groupby(ID_COL, sort=False))

#     risks = []
#     for L in TRACE_LENGTHS:
#         correct = 0
#         total = 0
#         used_traces = 0

#         for _, g in test_groups:
#             if len(g) < L:
#                 continue
#             used_traces += 1

#             gL = g.iloc[:L]  # first L points
#             X = gL[X_COLS].to_numpy(dtype=float)
#             y_true = gL[Y_COL].to_numpy(dtype=int)

#             prev_preds = []
#             # sequentially predict 1..L
#             for t in range(L):
#                 p_obs = knn_proba_full(knn, scaler, X[t], all_states)

#                 if t == 0:
#                     # L=1 concept: argmax Pr(s|o1)
#                     pred = int(all_states[np.argmax(p_obs)])
#                 else:
#                     # use history to improve predicting the current point
#                     r = min(R, t)  # up to t previous predictions
#                     p_hist = history_transition_mix(P, prior, all_states, prev_preds, r)
#                     score = p_obs * p_hist
#                     if score.sum() <= 0:
#                         score = p_obs
#                     pred = int(all_states[np.argmax(score)])

#                 prev_preds.append(pred)

#             # Evaluate ONLY the L-th prediction (index L-1)
#             total += 1
#             if prev_preds[L - 1] == y_true[L - 1]:
#                 correct += 1

#         acc = (correct / total) if total > 0 else np.nan
#         risk = 1.0 - acc if total > 0 else np.nan
#         risks.append(risk)

#         print(f"[{name}] L={L:>3} traces_used={used_traces:>4}  BayesRisk={risk}")

#     return risks


# # ============================================================
# # RUN
# # ============================================================
# if __name__ == "__main__":
#     df_plm  = pd.read_csv(PLM_CSV)
#     df_psm  = pd.read_csv(PSM_CSV)
#     df_psmi = pd.read_csv(PSMI_CSV)

#     risks_plm  = bayes_risk_vs_trace_length(df_plm,  "PLM")
#     risks_psm  = bayes_risk_vs_trace_length(df_psm,  "PSM")
#     risks_psmi = bayes_risk_vs_trace_length(df_psmi, "PSM-I")

#     print("\n==== FINAL OUTPUT ====")
#     print("trace_lengths =", TRACE_LENGTHS)
#     print("plm  =", risks_plm)
#     print("psm  =", risks_psm)
#     print("psmi =", risks_psmi)


In [9]:
# import numpy as np
# import pandas as pd
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.preprocessing import StandardScaler

# # ============================================================
# # EDIT THESE PATHS
# # ============================================================
# PLM_CSV  = r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1_copy_data_5\plm\grid40\merged_geotrace_plm_delta_3_Noisy_threshold_5_eps05_grid40.csv"
# PSM_CSV  = r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1_copy_data_5\psm\grid40\merged_geotrace_psm_delta_3_Noisy_threshold_5_eps05_grid40.csv"
# PSMI_CSV = r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1_copy_data_5\psmi\grid40\merged_geotrace_psmi_delta_3_Noisy_threshold_5_eps05_grid40.csv"

# # ============================================================
# # EDIT THESE COLUMN NAMES IF NEEDED
# # ============================================================
# ID_COL   = "identifier"   # trajectory/user id
# TIME_COL = None           # e.g., "timestamp" if you have it; else keep None

# X_COLS   = ["perturbed_latitude", "perturbed_longitude"]  # attacker observes these
# Y_COL    = "location_id"                                 # attacker predicts this

# # ============================================================
# # SETTINGS YOU CONTROL
# # ============================================================
# KNN_K = 5           # <-- YOU choose k here
# TEST_RATIO = 0.20
# SEED = 42

# TRACE_LENGTHS = [1, 2, 3, 5, 10, 20, 30]
# R_HISTORY = 10       # use up to last R predicted states for history term
# ALPHA_DECAY = 0.7    # exponential decay for history weights (recent states matter more)
# SMOOTH = 1e-6        # smoothing for transition probabilities
# USE_STANDARDIZE = True


# # ============================================================
# # UTILS
# # ============================================================
# def check_columns(df, required, name):
#     missing = [c for c in required if c not in df.columns]
#     if missing:
#         raise ValueError(f"[{name}] Missing columns: {missing}\nAvailable columns: {list(df.columns)}")

# def split_by_identifier(df, id_col, test_ratio, seed):
#     ids = df[id_col].dropna().unique()
#     rng = np.random.default_rng(seed)
#     rng.shuffle(ids)
#     n_test = int(np.ceil(len(ids) * test_ratio))
#     test_ids = set(ids[:n_test])
#     train_df = df[~df[id_col].isin(test_ids)].copy()
#     test_df  = df[df[id_col].isin(test_ids)].copy()
#     return train_df, test_df

# def safe_standardize_fit_transform(X):
#     scaler = StandardScaler()
#     return scaler, scaler.fit_transform(X)

# def safe_standardize_transform(scaler, X):
#     return scaler.transform(X)

# def build_transition_probs(train_df):
#     """
#     Build P(next | prev) from training TRUE location_id sequences.
#     Returns:
#       all_states: sorted states
#       P: dict prev_state -> prob vector over all_states
#       prior: global prior vector over all_states
#     """
#     all_states = np.sort(train_df[Y_COL].dropna().unique()).astype(int)
#     idx = {s: i for i, s in enumerate(all_states)}
#     K = len(all_states)

#     counts = np.zeros((K, K), dtype=float)
#     prior_counts = np.zeros(K, dtype=float)

#     for _, g in train_df.groupby(ID_COL, sort=False):
#         if TIME_COL is not None and TIME_COL in g.columns:
#             g = g.sort_values(TIME_COL, kind="stable")

#         seq = g[Y_COL].to_numpy()
#         seq = seq[~pd.isna(seq)].astype(int)
#         if len(seq) == 0:
#             continue

#         for s in seq:
#             if s in idx:
#                 prior_counts[idx[s]] += 1

#         for t in range(len(seq) - 1):
#             a, b = seq[t], seq[t + 1]
#             if a in idx and b in idx:
#                 counts[idx[a], idx[b]] += 1

#     # smooth + normalize
#     counts += SMOOTH
#     probs = counts / counts.sum(axis=1, keepdims=True)

#     prior_counts += SMOOTH
#     prior = prior_counts / prior_counts.sum()

#     P = {prev: probs[idx[prev]] for prev in all_states}
#     return all_states, P, prior

# def history_mix(P, prior, all_states, prev_preds, r, alpha):
#     """
#     Pr(s | history) = weighted mixture of P(s | prev_pred)
#     with exponential decay weights (most recent gets highest weight).
#     """
#     if len(prev_preds) == 0:
#         return prior.copy()

#     use = prev_preds[-r:]
#     # weights for most recent first
#     w = np.array([alpha**i for i in range(len(use))], dtype=float)  # i=0 most recent
#     w /= w.sum()

#     p_hist = np.zeros(len(all_states), dtype=float)
#     # iterate most recent -> older
#     for weight, prev_state in zip(w, reversed(use)):
#         p_hist += weight * (P.get(prev_state, prior))

#     ssum = p_hist.sum()
#     return prior.copy() if ssum <= 0 else (p_hist / ssum)

# def knn_proba_full(knn, scaler, x_row, all_states):
#     """
#     Get Pr(s|o) aligned with all_states.
#     """
#     x = x_row.reshape(1, -1)
#     if scaler is not None:
#         x = safe_standardize_transform(scaler, x)

#     p = knn.predict_proba(x)[0]
#     classes = knn.classes_.astype(int)

#     p_full = np.zeros(len(all_states), dtype=float)
#     cls_to_pos = {c: i for i, c in enumerate(classes)}
#     for i, s in enumerate(all_states):
#         j = cls_to_pos.get(s, None)
#         if j is not None:
#             p_full[i] = p[j]

#     ssum = p_full.sum()
#     if ssum <= 0:
#         p_full[:] = 1.0 / len(all_states)
#     else:
#         p_full /= ssum
#     return p_full

# def train_knn(train_df, k_user):
#     X = train_df[X_COLS].to_numpy(dtype=float)
#     y = train_df[Y_COL].to_numpy(dtype=int)

#     # drop rows with NaNs in X or y
#     mask = np.isfinite(X).all(axis=1) & np.isfinite(y)
#     X, y = X[mask], y[mask]

#     if len(X) == 0:
#         raise ValueError("Training data became empty after removing NaNs.")

#     scaler = None
#     if USE_STANDARDIZE:
#         scaler, X = safe_standardize_fit_transform(X)

#     # IMPORTANT: k must be <= number of training samples
#     k = int(k_user)
#     k = max(1, min(k, len(X)))

#     knn = KNeighborsClassifier(n_neighbors=k, weights="distance", n_jobs=-1)
#     knn.fit(X, y)
#     return knn, scaler, k

# def bayes_risk_vs_L(df, name):
#     check_columns(df, [ID_COL, Y_COL] + X_COLS, name)

#     # Split by trajectory (no leakage)
#     train_df, test_df = split_by_identifier(df, ID_COL, TEST_RATIO, SEED)

#     # Optional: sort within each identifier if time exists
#     if TIME_COL is not None and TIME_COL in df.columns:
#         train_df = train_df.sort_values([ID_COL, TIME_COL], kind="stable")
#         test_df  = test_df.sort_values([ID_COL, TIME_COL], kind="stable")

#     # Train kNN + build transitions
#     knn, scaler, k_used = train_knn(train_df, KNN_K)
#     all_states, P, prior = build_transition_probs(train_df)

#     print(f"[{name}] train_rows={len(train_df)} test_rows={len(test_df)} kNN_k_used={k_used} states={len(all_states)}")

#     risks = []
#     groups = list(test_df.groupby(ID_COL, sort=False))

#     for L in TRACE_LENGTHS:
#         correct_L = 0
#         total_L = 0
#         used_traces = 0

#         for _, g in groups:
#             if len(g) < L:
#                 continue
#             used_traces += 1

#             gL = g.iloc[:L]
#             X = gL[X_COLS].to_numpy(dtype=float)
#             y_true = gL[Y_COL].to_numpy(dtype=int)

#             # sequential prediction 1..L
#             prev_preds = []
#             for t in range(L):
#                 p_obs = knn_proba_full(knn, scaler, X[t], all_states)

#                 if t == 0:
#                     pred = int(all_states[np.argmax(p_obs)])
#                 else:
#                     r = min(R_HISTORY, t)
#                     p_hist = history_mix(P, prior, all_states, prev_preds, r, ALPHA_DECAY)
#                     score = p_obs * p_hist
#                     if score.sum() <= 0:
#                         score = p_obs
#                     pred = int(all_states[np.argmax(score)])

#                 prev_preds.append(pred)

#             # Evaluate ONLY the L-th point (your exact concept)
#             total_L += 1
#             if prev_preds[L - 1] == y_true[L - 1]:
#                 correct_L += 1

#         acc = (correct_L / total_L) if total_L > 0 else np.nan
#         risk = 1.0 - acc if total_L > 0 else np.nan
#         risks.append(risk)
#         print(f"[{name}] L={L:>3} traces_used={used_traces:>4} BayesRisk={risk}")

#     return risks


# # ============================================================
# # RUN
# # ============================================================
# if __name__ == "__main__":
#     df_plm  = pd.read_csv(PLM_CSV)
#     df_psm  = pd.read_csv(PSM_CSV)
#     df_psmi = pd.read_csv(PSMI_CSV)

#     risks_plm  = bayes_risk_vs_L(df_plm,  "PLM")
#     risks_psm  = bayes_risk_vs_L(df_psm,  "PSM")
#     risks_psmi = bayes_risk_vs_L(df_psmi, "PSM-I")

#     print("\n==== FINAL LISTS ====")
#     print("trace_lengths =", TRACE_LENGTHS)
#     print("plm  =", risks_plm)
#     print("psm  =", risks_psm)
#     print("psmi =", risks_psmi)


[PLM] train_rows=249170 test_rows=69685 kNN_k_used=5 states=259
[PLM] L=  1 traces_used=   8 BayesRisk=0.0
[PLM] L=  2 traces_used=   8 BayesRisk=0.0
[PLM] L=  3 traces_used=   8 BayesRisk=0.0
[PLM] L=  5 traces_used=   8 BayesRisk=0.0
[PLM] L= 10 traces_used=   8 BayesRisk=0.125
[PLM] L= 20 traces_used=   8 BayesRisk=0.0
[PLM] L= 30 traces_used=   8 BayesRisk=0.125
[PSM] train_rows=249170 test_rows=69685 kNN_k_used=5 states=259
[PSM] L=  1 traces_used=   8 BayesRisk=0.0
[PSM] L=  2 traces_used=   8 BayesRisk=0.0
[PSM] L=  3 traces_used=   8 BayesRisk=0.0
[PSM] L=  5 traces_used=   8 BayesRisk=0.0
[PSM] L= 10 traces_used=   8 BayesRisk=0.0
[PSM] L= 20 traces_used=   8 BayesRisk=0.125
[PSM] L= 30 traces_used=   8 BayesRisk=0.25
[PSM-I] train_rows=249170 test_rows=69685 kNN_k_used=5 states=259
[PSM-I] L=  1 traces_used=   8 BayesRisk=0.0
[PSM-I] L=  2 traces_used=   8 BayesRisk=0.0
[PSM-I] L=  3 traces_used=   8 BayesRisk=0.0
[PSM-I] L=  5 traces_used=   8 BayesRisk=0.0
[PSM-I] L= 10 tra

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# =========================
# CONFIG (EDIT)
# =========================
KNN_K = 5                 # YOU choose k
NUM_RUNS = 1            # like your num_runs (averaging randomness)
TEST_RATIO = 0.20
SEED0 = 42

TRACE_LENGTHS = [1,  5, 10, 20]   # evaluate these L values

ID_COL = "identifier"
X_COLS = ["perturbed_latitude", "perturbed_longitude"]
Y_COL  = "location_id"

USE_STANDARDIZE = True
SMOOTH = 1e-6             # smoothing for transition matrix


# =========================
# HELPERS
# =========================
def split_by_identifier(df, id_col, test_ratio, seed):
    ids = df[id_col].dropna().unique()
    rng = np.random.default_rng(seed)
    rng.shuffle(ids)
    n_test = int(np.ceil(len(ids) * test_ratio))
    test_ids = set(ids[:n_test])
    train_df = df[~df[id_col].isin(test_ids)].copy()
    test_df  = df[df[id_col].isin(test_ids)].copy()
    return train_df, test_df

def train_knn(train_df, k_user):
    X = train_df[X_COLS].to_numpy(dtype=float)
    y = train_df[Y_COL].to_numpy(dtype=int)

    # drop NaNs
    ok = np.isfinite(X).all(axis=1) & np.isfinite(y)
    X, y = X[ok], y[ok]

    scaler = None
    if USE_STANDARDIZE:
        scaler = StandardScaler()
        X = scaler.fit_transform(X)

    k = int(k_user)
    k = max(1, min(k, len(X)))  # prevent k > #train_samples

    knn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1, weights="distance")
    knn.fit(X, y)
    return knn, scaler, k

def knn_proba_full(knn, scaler, x_row, all_states):
    x = x_row.reshape(1, -1)
    if scaler is not None:
        x = scaler.transform(x)

    p = knn.predict_proba(x)[0]
    classes = knn.classes_.astype(int)

    p_full = np.zeros(len(all_states), dtype=float)
    cls_to_pos = {c: i for i, c in enumerate(classes)}
    for i, s in enumerate(all_states):
        j = cls_to_pos.get(s, None)
        if j is not None:
            p_full[i] = p[j]

    ssum = p_full.sum()
    if ssum <= 0:
        p_full[:] = 1.0 / len(all_states)
    else:
        p_full /= ssum
    return p_full

def build_transition_matrix(train_df):
    """
    First-order Markov transition: P(next | prev)
    learned from TRAIN traces.
    """
    all_states = np.sort(train_df[Y_COL].dropna().unique()).astype(int)
    idx = {s: i for i, s in enumerate(all_states)}
    K = len(all_states)

    counts = np.zeros((K, K), dtype=float)

    for _, g in train_df.groupby(ID_COL, sort=False):
        seq = g[Y_COL].to_numpy()
        seq = seq[~pd.isna(seq)].astype(int)
        if len(seq) < 2:
            continue
        for t in range(len(seq) - 1):
            a, b = seq[t], seq[t + 1]
            if a in idx and b in idx:
                counts[idx[a], idx[b]] += 1

    counts += SMOOTH
    probs = counts / counts.sum(axis=1, keepdims=True)

    # dict: prev_state -> prob vector over all_states
    P = {prev: probs[idx[prev]] for prev in all_states}
    return all_states, P

def sequential_predict_prefix(gL, knn, scaler, all_states, P):
    """
    Predict s1..sL sequentially:
      - t=1: argmax Pr(s|o1)
      - t>=2: argmax Pr(s|ot) * P(s | s_hat_{t-1})
    Return predicted list length L.
    """
    X = gL[X_COLS].to_numpy(dtype=float)
    preds = []

    for t in range(len(gL)):
        p_obs = knn_proba_full(knn, scaler, X[t], all_states)

        if t == 0:
            pred = int(all_states[np.argmax(p_obs)])
        else:
            prev = preds[t - 1]
            p_tr = P.get(prev, np.ones(len(all_states)) / len(all_states))
            score = p_obs * p_tr
            if score.sum() <= 0:
                score = p_obs
            pred = int(all_states[np.argmax(score)])

        preds.append(pred)
    return preds


def bayes_risk_vs_trace_length(df, name):
    # quick column check
    for c in [ID_COL, Y_COL] + X_COLS:
        if c not in df.columns:
            raise ValueError(f"[{name}] Missing column '{c}'. Available: {list(df.columns)}")

    # store risks for each L across runs
    risk_runs = {L: [] for L in TRACE_LENGTHS}

    for run in range(NUM_RUNS):
        seed = SEED0 + run

        # split by identifier (no leakage)
        train_df, test_df = split_by_identifier(df, ID_COL, TEST_RATIO, seed)

        # train kNN + transitions
        knn, scaler, k_used = train_knn(train_df, KNN_K)
        all_states, P = build_transition_matrix(train_df)

        # group test traces
        groups = list(test_df.groupby(ID_COL, sort=False))

        for L in TRACE_LENGTHS:
            correct = 0
            total = 0

            for _, g in groups:
                if len(g) < L:
                    continue

                gL = g.iloc[:L]              # first L points
                y_true_L = int(gL[Y_COL].iloc[L - 1])

                preds = sequential_predict_prefix(gL, knn, scaler, all_states, P)
                y_pred_L = preds[L - 1]

                total += 1
                if y_pred_L == y_true_L:
                    correct += 1

            if total == 0:
                risk_runs[L].append(np.nan)
            else:
                acc = correct / total
                risk_runs[L].append((1 - acc) * 100.0)

    # average across runs
    risks_avg = []
    for L in TRACE_LENGTHS:
        arr = np.array(risk_runs[L], dtype=float)
        risks_avg.append(float(np.nanmean(arr)))

    print(f"[{name}] done. (k={KNN_K}, runs={NUM_RUNS})")
    return risks_avg


# =========================
# RUN FOR THREE MECHANISMS
# =========================
if __name__ == "__main__":
    # Replace these with your merged CSV paths
    PLM_PATH  = r"D:\backup\location_privacy_final\collected\machine_learning\attack1\laplace\200\merged_laplace_0.1_encoded_200.csv"
    #PSM_PATH  = r"D:\backup\location_privacy_final\collected\machine_learning\attack1\laplace\400\merged_laplace_0.1_encoded_400.csv"
    PSMI_PATH = r"D:\backup\location_privacy_final\collected\machine_learning\attack1\our_bl_50_delta_5\200\merged_our_bl_50_delta_5_0.1_encoded_200.csv"

    df_plm  = pd.read_csv(PLM_PATH)
    #df_psm  = pd.read_csv(PSM_PATH)
    df_psmi = pd.read_csv(PSMI_PATH)

    risks_plm  = bayes_risk_vs_trace_length(df_plm,  "PLM")
    #risks_psm  = bayes_risk_vs_trace_length(df_psm,  "PSM")
    risks_psmi = bayes_risk_vs_trace_length(df_psmi, "PSM-I")

    print("\ntrace_lengths =", TRACE_LENGTHS)
    print("plm  =", risks_plm)
    #print("psm  =", risks_psm)
    print("psmi =", risks_psmi)


In [24]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# =========================
# CONFIG (EDIT)
# =========================
NUM_RUNS   = 20
TEST_RATIO = 0.20
SEED0      = 42

TRACE_LENGTHS = [1, 5, 10, 20]

ID_COL = "identifier"
X_COLS = ["perturbed_latitude", "perturbed_longitude"]
Y_COL  = "location_id"

USE_STANDARDIZE = True
SMOOTH = 1e-6  # smoothing for transition matrix

# Auto-k selection settings
VAL_RATIO = 0.20
K_CANDIDATES = list(range(1, 52, 2))  # odd k's 1..51


# =========================
# HELPERS
# =========================
def split_by_identifier(df, id_col, test_ratio, seed):
    ids = df[id_col].dropna().unique()
    rng = np.random.default_rng(seed)
    rng.shuffle(ids)
    n_test = int(np.ceil(len(ids) * test_ratio))
    test_ids = set(ids[:n_test])
    train_df = df[~df[id_col].isin(test_ids)].copy()
    test_df  = df[df[id_col].isin(test_ids)].copy()
    return train_df, test_df


def make_xy(df):
    X = df[X_COLS].to_numpy(dtype=float)
    y = df[Y_COL].to_numpy(dtype=int)
    ok = np.isfinite(X).all(axis=1) & np.isfinite(y)
    return X[ok], y[ok]


def train_knn_auto_k(train_df, id_col, seed):
    """
    Select k via validation split by identifier (no leakage), then train final kNN on full train_df.
    Returns: knn, scaler, best_k
    """
    # ---- sub-train / val split by identifier ----
    ids = train_df[id_col].dropna().unique()
    rng = np.random.default_rng(seed)
    rng.shuffle(ids)
    n_val = int(np.ceil(len(ids) * VAL_RATIO))
    val_ids = set(ids[:n_val])

    sub_train = train_df[~train_df[id_col].isin(val_ids)].copy()
    val_df    = train_df[train_df[id_col].isin(val_ids)].copy()

    X_tr, y_tr = make_xy(sub_train)
    X_va, y_va = make_xy(val_df)

    if len(X_tr) == 0 or len(X_va) == 0:
        raise ValueError("Empty sub-train or val after NaN filtering. Check data/columns.")

    scaler = None
    if USE_STANDARDIZE:
        scaler = StandardScaler()
        X_tr = scaler.fit_transform(X_tr)
        X_va = scaler.transform(X_va)

    # clamp candidates so k <= #sub_train
    max_k = len(X_tr)
    k_candidates = [k for k in K_CANDIDATES if k <= max_k]
    if not k_candidates:
        k_candidates = [max(1, min(5, max_k))]

    best_k = k_candidates[0]
    best_acc = -1.0

    for k in k_candidates:
        knn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1, weights="distance")
        knn.fit(X_tr, y_tr)
        acc = float((knn.predict(X_va) == y_va).mean())
        if acc > best_acc:
            best_acc = acc
            best_k = k

    # ---- retrain final model on full train_df with best_k ----
    X_full, y_full = make_xy(train_df)

    scaler_full = None
    if USE_STANDARDIZE:
        scaler_full = StandardScaler()
        X_full = scaler_full.fit_transform(X_full)

    best_k = max(1, min(best_k, len(X_full)))
    knn_final = KNeighborsClassifier(n_neighbors=best_k, n_jobs=-1, weights="distance")
    knn_final.fit(X_full, y_full)

    return knn_final, scaler_full, best_k


def knn_proba_full(knn, scaler, x_row, all_states):
    x = x_row.reshape(1, -1)
    if scaler is not None:
        x = scaler.transform(x)

    p = knn.predict_proba(x)[0]
    classes = knn.classes_.astype(int)

    p_full = np.zeros(len(all_states), dtype=float)
    cls_to_pos = {c: i for i, c in enumerate(classes)}
    for i, s in enumerate(all_states):
        j = cls_to_pos.get(s, None)
        if j is not None:
            p_full[i] = p[j]

    ssum = p_full.sum()
    if ssum <= 0:
        p_full[:] = 1.0 / len(all_states)
    else:
        p_full /= ssum
    return p_full


def build_transition_matrix(train_df):
    all_states = np.sort(train_df[Y_COL].dropna().unique()).astype(int)
    idx = {s: i for i, s in enumerate(all_states)}
    K = len(all_states)

    counts = np.zeros((K, K), dtype=float)

    for _, g in train_df.groupby(ID_COL, sort=False):
        seq = g[Y_COL].to_numpy()
        seq = seq[~pd.isna(seq)].astype(int)
        if len(seq) < 2:
            continue
        for t in range(len(seq) - 1):
            a, b = seq[t], seq[t + 1]
            if a in idx and b in idx:
                counts[idx[a], idx[b]] += 1

    counts += SMOOTH
    probs = counts / counts.sum(axis=1, keepdims=True)

    P = {prev: probs[idx[prev]] for prev in all_states}
    return all_states, P


def sequential_predict_prefix(gL, knn, scaler, all_states, P):
    X = gL[X_COLS].to_numpy(dtype=float)
    preds = []

    for t in range(len(gL)):
        p_obs = knn_proba_full(knn, scaler, X[t], all_states)

        if t == 0:
            pred = int(all_states[np.argmax(p_obs)])
        else:
            prev = preds[t - 1]
            p_tr = P.get(prev, np.ones(len(all_states)) / len(all_states))
            score = p_obs * p_tr
            if score.sum() <= 0:
                score = p_obs
            pred = int(all_states[np.argmax(score)])

        preds.append(pred)
    return preds


def bayes_risk_vs_trace_length(df, name):
    for c in [ID_COL, Y_COL] + X_COLS:
        if c not in df.columns:
            raise ValueError(f"[{name}] Missing column '{c}'. Available: {list(df.columns)}")

    risk_runs = {L: [] for L in TRACE_LENGTHS}

    for run in range(NUM_RUNS):
        seed = SEED0 + run

        train_df, test_df = split_by_identifier(df, ID_COL, TEST_RATIO, seed)

        # AUTO-select k here
        knn, scaler, k_used = train_knn_auto_k(train_df, ID_COL, seed)
        all_states, P = build_transition_matrix(train_df)

        groups = list(test_df.groupby(ID_COL, sort=False))

        for L in TRACE_LENGTHS:
            correct = 0
            total = 0

            for _, g in groups:
                if len(g) < L:
                    continue

                gL = g.iloc[:L]
                y_true_L = int(gL[Y_COL].iloc[L - 1])

                preds = sequential_predict_prefix(gL, knn, scaler, all_states, P)
                y_pred_L = preds[L - 1]

                total += 1
                if y_pred_L == y_true_L:
                    correct += 1

            if total == 0:
                risk_runs[L].append(np.nan)
            else:
                acc = correct / total
                risk_runs[L].append((1 - acc) * 100.0)

        print(f"[{name}] run={run} selected_k={k_used}")

    risks_avg = []
    for L in TRACE_LENGTHS:
        arr = np.array(risk_runs[L], dtype=float)
        risks_avg.append(float(np.nanmean(arr)))

    print(f"[{name}] done. runs={NUM_RUNS}")
    return risks_avg


# =========================
# RUN
# =========================
if __name__ == "__main__":
    PLM_PATH  = r"D:\backup\location_privacy_final\collected\machine_learning\attack50\laplace\400\merged_laplace_0.1_encoded_400.csv"
    PSMI_PATH = r"D:\backup\location_privacy_final\collected\machine_learning\attack50\our_bl_50_delta_5\400\merged_our_bl_50_delta_5_0.1_encoded_400.csv"

    df_plm  = pd.read_csv(PLM_PATH)
    df_psmi = pd.read_csv(PSMI_PATH)

    risks_plm  = bayes_risk_vs_trace_length(df_plm,  "PLM")
    risks_psmi = bayes_risk_vs_trace_length(df_psmi, "PSM-I")

    print("\ntrace_lengths =", TRACE_LENGTHS)
    print("plm  =", risks_plm)
    print("psmi =", risks_psmi)


[PLM] run=0 selected_k=31
[PLM] run=1 selected_k=39
[PLM] run=2 selected_k=39
[PLM] run=3 selected_k=17
[PLM] run=4 selected_k=17
[PLM] run=5 selected_k=9
[PLM] run=6 selected_k=33
[PLM] run=7 selected_k=21
[PLM] run=8 selected_k=31
[PLM] run=9 selected_k=35
[PLM] run=10 selected_k=43
[PLM] run=11 selected_k=37
[PLM] run=12 selected_k=21
[PLM] run=13 selected_k=27
[PLM] run=14 selected_k=45
[PLM] run=15 selected_k=19
[PLM] run=16 selected_k=33
[PLM] run=17 selected_k=43
[PLM] run=18 selected_k=15
[PLM] run=19 selected_k=33
[PLM] done. runs=20
[PSM-I] run=0 selected_k=29
[PSM-I] run=1 selected_k=45
[PSM-I] run=2 selected_k=51
[PSM-I] run=3 selected_k=49
[PSM-I] run=4 selected_k=51
[PSM-I] run=5 selected_k=23
[PSM-I] run=6 selected_k=41
[PSM-I] run=7 selected_k=51
[PSM-I] run=8 selected_k=45
[PSM-I] run=9 selected_k=51
[PSM-I] run=10 selected_k=41
[PSM-I] run=11 selected_k=35
[PSM-I] run=12 selected_k=51
[PSM-I] run=13 selected_k=35
[PSM-I] run=14 selected_k=35
[PSM-I] run=15 selected_k=